In [1]:
from src import network, replay_memory_agent, deep_q_agent, epsi_greedy
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt

env = gym.make("CartPole-v0")

[2017-12-10 15:30:37,748] Making new env: CartPole-v0


In [2]:
deepq_session = tf.Session()

q_nn = network(input_states=4,
               num_layers=2,
               nurons_list=[10, 10],
               output_states=2,
               session=deepq_session)

target_nn = network(input_states=4,
                    num_layers=2,
                    nurons_list=[10, 10],
                    output_states=2,
                    session=deepq_session)

target_nn.set_weights(q_nn.get_weights())

In [3]:
replay1 = replay_memory_agent(4, 5000)
dqn_controller = deep_q_agent(action_value_model=q_nn,
                              target_model=target_nn,
                              states_len=4,
                              replay_memory=replay1)

In [4]:
# Book keeping
avg_reward_episodes = []

In [ ]:
NUM_EPISODES = 10000
# Exploration decay
epsilon = np.linspace(0.20, 0.001, NUM_EPISODES+10)
# Global time for target model update
global_time = 0

In [ ]:
for episodes in range(0, NUM_EPISODES):
    
    # Initial State
    state = env.reset()
    done=False
    # Clear the reward buffer
    rewards = []
    
    while not(done):
        global_time += 1

        # Reshape the state
        state = np.asarray(state)
        state = state.reshape(1,4)

        # Pick a action based on the state
        q_values = q_nn.predict_on_batch(state)
        action = epsi_greedy([0, 1], q_values, epsilon[episodes])

        # Implement action and observe the reward signal
        state_new, reward, done, _ = env.step(action)
        rewards.append(reward)

        state_new = np.asarray(state_new)
        state_new = state_new.reshape(1,4)

        # Update the replay memory
        replay1.replay_memory_update(state, state_new, reward, action, done)

        # Train
        update = True if global_time%5000==0 else False
        dqn_controller.train_q(update)

        state = state_new

    avg_reward_episodes.append(sum(rewards))
    
    if episodes%200 == 0:
        print(sum(rewards), "Episode Count :" ,episodes)
        q_nn.save_weights("model"+str(episodes))
        np.save("sum_rewards"+str(episodes), avg_reward_episodes)

11.0 Episode Count : 0
Weights Saved to  model0
8.0 Episode Count : 200
Weights Saved to  model200
8.0 Episode Count : 400
Weights Saved to  model400
9.0 Episode Count : 600
Weights Saved to  model600
10.0 Episode Count : 800
Weights Saved to  model800
22.0 Episode Count : 1000
Weights Saved to  model1000
14.0 Episode Count : 1200
Weights Saved to  model1200
12.0 Episode Count : 1400
Weights Saved to  model1400
8.0 Episode Count : 1600
Weights Saved to  model1600
12.0 Episode Count : 1800
Weights Saved to  model1800
10.0 Episode Count : 2000
Weights Saved to  model2000
12.0 Episode Count : 2200
Weights Saved to  model2200
9.0 Episode Count : 2400
Weights Saved to  model2400
13.0 Episode Count : 2600
Weights Saved to  model2600
9.0 Episode Count : 2800
Weights Saved to  model2800
10.0 Episode Count : 3000
Weights Saved to  model3000
9.0 Episode Count : 3200
Weights Saved to  model3200
9.0 Episode Count : 3400
Weights Saved to  model3400
9.0 Episode Count : 3600
Weights Saved to  model36

In [ ]:
print("hi")